# dp100_14 パイプライン

## パイプラインのステップ

- パイプラインは、タスクを実行する1つ以上の"ステップ"で構成される
- AzureMLパイプラインでの一般的なステップは次の通り。
    - PythonScriptStep:
        - 指定されたPythonスクリプトを実行する
    - DataTransferStep:
        - Azure Data Factoryを使用してデータストア間でデータコピー
    - DatabriksStep:
        - Databricksクラスターでノートブック、スクリプト、またはコンパイル済みJARを実行
    - AdlaStep:
        - Azure Data Lake AnalyticsでU-SQLジョブを実行する
    - ParallelRunStep:
        - 複数のコンピューティングノードで分散タスクとしてPythonスクリプトを実行
- 作成するには、各ステップを定義してからパイプラインを作成する必要がある
    - ステップの作成は`step = PythonScriptStep(name,source_directory,script_name,compute_target)`
    - パイプラインの割り当ては`train_pipeline = Pipeline(workspace, steps = [step, ...])`
        - エクスペリメントとして実行 `experiment.submit(train_pipeline)`
    
## パイプラインのステップ間のデータやり取り

- `PipelineData`でステップ間のデータのやり取りを行う
    - パイプラインデータを`prepped_data = PipelineData('prepped',  datastore=data_store)`
    - `arguments`にパイプラインデータを紐付け
    - スクリプト自体はローカルフォルダ-のように使用できる
- パイプラインはステップを基本的に再利用する
    - パイプラインの再利用を無効にしたい場合、`PythonScriptStep(allow_reuse = False)`
    - すべてのステップを強制的に実行したい場合、`experiment.submit(regenerate_outputs=True)`を設定

## パイプラインの発行

- RESTエンドポイントの作成ができる
- 発行するには、`published_pipeline = pipeline.publish(name, description, version)`
- エクスペリメントが正常に実行されたら、`run.publish_pipeline(name, description, version)`でも発行できる
- エンドポイントのURIを`published_pipeline.endpoint`で取得できる

## パイプラインパラメータの使用

- パイプラインのパラメータを定義したい場合、`reg_param = PipelineParameter(name='reg_rate', default_value=0.01)`をしてから、  
`arguments`にてパラメータを追加すればできる

## パイプラインのスケジュール設定

- `ScheduleRecurrence`で頻度を定義し、`Schedule.create`で作成する必要がある
- データの変更をパイプライン実行トリガーにしたい場合、`ath_on_datastore='data/training'`で指定パスを監視する`Schedule`を作成する

## よくわからんやつ

```
# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env
```